# Introduction

In this notebook, we will explore the OpenAI library. We will also acquire data in different ways, such that we can inform our queries.

The setup is not different from what we have done before.

In [ ]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

In [ ]:
import sys
sys.path.append('../../05_src/')

In [ ]:
from utils.logger import get_logger
_logs = get_logger(__name__)

In [ ]:
from openai import OpenAI
client = OpenAI()

2. We create an API call and store the result in the variable `response`. Notice that the call specifies the model that we want to use, as well as an input. This is a simple call, the [responses API can handle more complex calls](https://platform.openai.com/docs/api-reference/responses).

# Longer Context

Thus far, we have manually created our context and prompt. We can use any of Python text manipulation tools to create documents. 

For instance, in the code below, we will use the request library to download a book from Project Guttenberg and include it as part of the context. Some useful links are:

- Python's [request](https://pypi.org/project/requests/) library.
- Python's request library [documentation](https://requests.readthedocs.io/en/latest/).


In [ ]:
import requests
file_url = 'https://www.gutenberg.org/cache/epub/223/pg223.txt'
book = requests.get(file_url)

The code below shows how to obtain the headers in the response. 

In [ ]:
dict(book.headers)

We can also obtain the payload using `book.text` or `book.content`.

In [ ]:
print(book.text)

With added content, we can also write more interesting prompts. Notice the use of Python's f-strings or formatted strings. 

When using formatted strings, remember two things:

+ Formatted strings are prefixed with f:  `f'This is a formatted string: {some_variable}.`
+ You can enclose variable names in `{...}` and their values will appear in formatted strings: `f'The variable value is {variable}.`

In [ ]:
prompt = f"""
    You are a literary historian. 
    Given the following context from a book, do the following:
    
    1. Identify the book's title and author.
    2. Determine how many stories are included in the book.
    3. Summarize concisely in no more than 5 paragraphs the first story of the book called "The Absence of Mr Glass".
        
    The book is the following: 
    <book>
    {book.text}
    </book>

    Provide your response in the following format:
    Title: <title>
    Author: <author>
    Number of Stories: <number_of_stories>
    Summary: <summary>
"""

In [ ]:
response = client.responses.create(
    model = 'gpt-4o',
    input = prompt,
)


In [ ]:
print(response.output_text)

# Adding a Developer Prompt

Using the OpenAI library, we can specify prompts for different roles:

+ *Developer* messages are instructions provided by the application developer, prioritized ahead of user messages. In previous iterations of the API, these were called *System* messages.
+ *User* messages are instructions provided by an end user, prioritized below developer messages.
+ *Assistant* messages are generated by the model.

In [ ]:
system_prompt = "You are an anthropologist specializing in ancient fables with the vocabulary of Uncle Rogers, the TV character."

In [ ]:
the_ant = """
Ants were once men and made their living by tilling the soil. But, not content with the results of their own work, they were always casting longing eyes upon the crops and fruits of their neighbours, which they stole, whenever they got the chance, and added to their own store. At last their covetousness made Jupiter so angry that he changed them into Ants. But, though their forms were changed, their nature remained the same: and so, to this day, they go about among the cornfields and gather the fruits of others’ labour, and store them up for their own use.
"""

prompt = f"""
    Please, extract the main message of this fable written by Aesop.
    The fable is the following: 
    <fable>
    {the_ant}
    </fable>
"""

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    instructions = system_prompt,
    input = prompt,
)

print(response.output_text)